# **GCD_v1 algorithm in with Vlad's lib**
[The detailed algorithm](https://web.eecs.umich.edu/~weimerw/2015-4610/scottcd/10a_thfnd.pdf)
[Helped Wiki](https://en.wikipedia.org/wiki/Lambda_calculus)

1) gcd ≡ λa.λb.(if (equal a b) a (if (greater than a b) (gcd (minus a b) b) (gcd (Minus b a) a)))
2) gcd ≡ (λg .λa.λb.(if (equal a b) a (if (greater than a b) (g (minus a b) b) (g (minus b a) a)))) gcd
3) gcd ≡ f gcd
4) λg .λa.λb.(if (equal a b) a (if (greater than a b) (g (minus a b) b) (g (minus b a) a)))
5) gcd ≡ (λh.(λx.h(x x)) (λx.h(x x)))
        (λg .λa.λb.(if (equal a b) a
        (if (greater than a b) (g (minus a b) b) (g (minus b a) a))))
6) Y ≡ (λh.(λx.h(x x)) (λx.h(x x)))
7) gcd ≡ Y
        (λg .λa.λb.(if (equal a b) a
        (if (greater than a b) (g (minus a b) b) (g (minus b a) a))))


In [1]:
import sys
import time

sys.path.append("../")
from calculus.term import *
from calculus.strategy import *
from calculus.advanced_terms import *

gcd ≡ (Y
        (λg.λa.λb.(ite (equal a b) a
        (ite (greater_than a b) (g (minus a b) b) (g (minus b a) a))))
        )

In [ ]:
def pred_simple_term():
    n, f, x, g, h, u = Var(), Var(), Var(), Var(), Var(), Var()
    n_, f_, x_, g_, h_, u_ = Atom(n), Atom(f), Atom(x), Atom(g), Atom(h), Atom(u)

    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(
        n_,
        Lambda(g, Lambda(h, App(h_, App(g_, f_)))),
        Lambda(u, x_),
        Lambda(u, u_)
    ))))

In [ ]:
def minus_term():
    m, n = Var(), Var()
    m_, n_ = Atom(m), Atom(n)
    return Lambda(m, Lambda(n, multi_app_term(n_, pred_simple_term(), m_)))

In [ ]:
strategy = RightmostInnermostStrategy()

term = App(pred_simple_term(), n_term(0))
print(term.normalize(strategy)[0].funky_str())

term = App(pred_simple_term(), n_term(1))
print(term.normalize(strategy)[0].funky_str())

term = App(pred_simple_term(), n_term(4))
print(term.normalize(strategy)[0].funky_str())

term = App(pred_simple_term(), n_term(10))
print(term.normalize(strategy)[0].funky_str())

In [ ]:
strategy = LeftmostOutermostStrategy()

term = multi_app_term(minus_term(), n_term(0), n_term(0))
print(term.normalize(strategy)[0].funky_str())

term = multi_app_term(minus_term(), n_term(3), n_term(0))
print(term.normalize(strategy)[0].funky_str())

term = multi_app_term(minus_term(), n_term(0), n_term(3))
print(term.normalize(strategy)[0].funky_str())

term = multi_app_term(minus_term(), n_term(4), n_term(2))
print(term.normalize(strategy)[0].funky_str())

gcd ≡ (Y
        (λg.λa.λb.(ite (equal a b) a
        (ite (greater_than a b) (g (minus a b) b) (g (minus b a) a) )))
        )


In [2]:
def gcd_v2():
    g, a, b = Var(), Var(), Var()
    g_, a_, b_ = Atom(g), Atom(a), Atom(b)
    equal_ab = multi_app_term(eq_term(), a_, b_)

    minus_ab = multi_app_term(g_, multi_app_term(substr_term(), a_, b_), b_)
    minus_ba = multi_app_term(g_, multi_app_term(substr_term(), b_, a_), a_)
    le_ba = multi_app_term(le_term(), b_, a_)
    inner_ite_term = multi_app_term(ite_term(), le_ba, minus_ab, minus_ba)

    ite_main_term = multi_app_term(ite_term(), equal_ab, a_, inner_ite_term)
    gcd_lambda = Lambda(g, Lambda(a, Lambda(b, ite_main_term)))
    return App(y_term(), gcd_lambda)

In [3]:
def test_gcd(var_0=0, var_1=0, strategy=LeftmostOutermostStrategy()):
    term_ = multi_app_term(gcd_v2(), n_term(var_0), n_term(var_1))
    res_term, steps = term_.normalize_no_lim(strategy)

    # print(term_.funky_str(), "\n\n")
    print(steps)
    print(res_term.funky_str())

In [4]:
test_gcd(0, 0, LeftmostOutermostStrategy())

37
(λw.(λq.q))


In [ ]:
test_gcd(0, 0, RightmostInnermostStrategy())

In [5]:
test_gcd(1, 1, LeftmostOutermostStrategy())

69
(λq.(λw.(q w)))


In [6]:
test_gcd(3, 6, LeftmostOutermostStrategy())

1034
(λw.(λq.(w (w (w q)))))


In [7]:
test_gcd(6, 3, LeftmostOutermostStrategy())

1034
(λq.(λw.(q (q (q w)))))


In [8]:
test_gcd(0, 5, LeftmostOutermostStrategy())

KeyboardInterrupt: 

In [ ]:
test_gcd(7, 0, LeftmostOutermostStrategy())

In [9]:
def gcd_v1_1():
    g, a, b = Var(), Var(), Var()
    g_, a_, b_ = Atom(g), Atom(a), Atom(b)
    equal_ab = multi_app_term(eq_term(), a_, b_)

    minus_ab = multi_app_term(g_, multi_app_term(substr_term(), a_, b_), b_)
    minus_ba = multi_app_term(g_, multi_app_term(substr_term(), b_, a_), a_)
    le_ba = multi_app_term(le_term(), b_, a_)
    inner_ite_term = multi_app_term(ite_term(), le_ba, minus_ab, minus_ba)

    ite_left_term = multi_app_term(ite_term(), equal_ab, a_, inner_ite_term)
    main_or_term = multi_app_term(or_term(), App(iszero_term(), a_), App(iszero_term(), b_))
    ite_main_term = multi_app_term(ite_term(), main_or_term, n_term(1), ite_left_term)
    gcd_lambda = Lambda(g, Lambda(a, Lambda(b, ite_main_term)))
    return App(y_term(), gcd_lambda)


def test_gcd_v1_1(var_0=0, var_1=0, strategy=LeftmostOutermostStrategy()):
    t_start = time.time()
    term_ = multi_app_term(gcd_v1_1(), n_term(var_0), n_term(var_1))

    res_term, steps = term_.normalize_no_lim(strategy)
    t_end = time.time() - t_start
    print(f"time spent: {t_end}")
    print(f"steps: {steps}")
    print(f"res term: {res_term}")

In [10]:
test_gcd_v1_1(0, 0, LeftmostOutermostStrategy())

time spent: 0.2920651435852051
steps: 23
res term: (λq.(λw.(q w)))


In [11]:
test_gcd_v1_1(1, 1, LeftmostOutermostStrategy())

time spent: 1.9239377975463867
steps: 89
res term: (λq.(λw.(q w)))


In [12]:
test_gcd_v1_1(0, 0, RightmostInnermostStrategy())

KeyboardInterrupt: 

In [13]:
test_gcd_v1_1(3, 6, LeftmostOutermostStrategy())

time spent: 21.101314067840576
steps: 1162
res term: (λw.(λq.(w (w (w q)))))


In [14]:
test_gcd_v1_1(6, 3, LeftmostOutermostStrategy())

time spent: 20.812254905700684
steps: 1162
res term: (λq.(λw.(q (q (q w)))))


In [15]:
test_gcd_v1_1(5, 3, LeftmostOutermostStrategy())

time spent: 112.59880447387695
steps: 4665
res term: (λq.(λw.(q w)))


In [16]:
test_gcd_v1_1(3, 5, LeftmostOutermostStrategy())

time spent: 118.19282841682434
steps: 4665
res term: (λq.(λw.(q w)))
